veri görselleştirme https://batuhandaz.medium.com/decision-tree-algoritmas%C4%B1-karar-a%C4%9Fac%C4%B1-machine-learning-78d856b1f457
https://ece-akdagli.medium.com/makine-%C3%B6%C4%9Frenmesinde-decision-tree-42a86502ee75
https://erdincuzun.com/makine_ogrenmesi/decision-tree-karar-agaci-id3-algoritmasi-classification-siniflama/

### Import Librarires And Dataset

In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#some settings to show data
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# target_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data")
target_url = ("datasets/abalone.data")
abalone = pd.read_csv(target_url)
headers = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight',
           'Rings']
abalone.columns = headers

### Analyze the Data

In [ ]:
abalone.head()

In [ ]:
abalone.describe()

In [ ]:
abalone.info()

**Get target value**

In [38]:
X = abalone.drop(columns="Rings")
y = abalone["Rings"]
X['Sex'] = abalone['Sex'].replace({'F':0,'M':1,'I':2})

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [40]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
1245,2,0.390,0.300,0.095,0.3265,0.1665,0.0575,0.0890
860,0,0.595,0.465,0.140,1.1130,0.5175,0.2440,0.3050
3675,0,0.600,0.500,0.160,1.1220,0.5095,0.2560,0.3090
1712,1,0.645,0.500,0.175,1.3375,0.5540,0.3080,0.4150
896,2,0.265,0.195,0.060,0.0920,0.0345,0.0250,0.0245
...,...,...,...,...,...,...,...,...
252,0,0.575,0.460,0.185,1.0940,0.4485,0.2170,0.3450
2020,1,0.525,0.410,0.130,0.6875,0.3435,0.1495,0.1765
3196,2,0.310,0.235,0.075,0.1515,0.0560,0.0315,0.0500
3268,0,0.595,0.465,0.145,1.1070,0.4020,0.2415,0.3100


In [ ]:
def is_numeric(x):
    return x == 'int' or x == 'float'
    # return int(x == 'int' or x == 'float')

In [ ]:
attribute_types = X_train.dtypes.apply(is_numeric)
attribute_types

In [ ]:
def build_dt(X, y, attribute_types, options=False):
    print("any implemantation")

In [ ]:
build_dt(X_train, y_train, attribute_types)

In [ ]:
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return f"Is {headers[self.column]} {condition} {self.value}"

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

In [ ]:
example = list(X_train.iloc[0])
q = Question(0, "M")
q.match(example)

In [ ]:
def partition(df, question):
    true_rows, false_rows = [], []
    for row in df.values:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [ ]:
true_rows, false_rows = partition(X_train, Question(0, 'M'))
true_rows

In [ ]:
def class_counts(rows):
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [ ]:
counts = class_counts(X_train.values)
print(counts)

In [ ]:
def gini(rows):
    # There are some ways, but I use gini impurity : https://medium.com/machine-learning-t%C3%BCrkiye/karar-agaclari-algoritmasi-b823c23997d0
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl ** 2
    return impurity